In [1]:
import pandas as pd
import folium

In [2]:
df=pd.read_csv('master_venue_categories_bc.csv')

In [3]:
fsa_latlon = df[['FSA','lat','lon']].groupby('FSA').agg('mean').reset_index()

In [4]:
fsa_income = pd.read_csv('census_2016_by_fsa/98-401-X2016046_English_CSV_data.csv').rename(columns={
                    'Member ID: Profile of Forward Sortation Areas (2247)':'DIM_ID',
                    'Dim: Sex (3): Member ID: [1]: Total - Sex':'Value',
                    'DIM: Profile of Forward Sortation Areas (2247)':'DIM'})
fsa_income= fsa_income[fsa_income['DIM_ID']==801]
fsa_income.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,DIM,DIM_ID,Notes: Profile of Forward Sortation Areas (2247),Value,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
800,2016,01,0,Canada,4.0,5.1,20000,01,Median total income of economic families in 20...,801,NaN,88306,...,...
3047,2016,A0A,2,A0A,4.2,9.1,0,01A0A,Median total income of economic families in 20...,801,NaN,80074,...,...
5294,2016,A0B,2,A0B,4.1,10.1,10,01A0B,Median total income of economic families in 20...,801,NaN,72917,...,...
7541,2016,A0C,2,A0C,4.1,8.6,0,01A0C,Median total income of economic families in 20...,801,NaN,62969,...,...
9788,2016,A0E,2,A0E,4.2,8.1,0,01A0E,Median total income of economic families in 20...,801,NaN,77716,...,...


In [5]:
fsa_facts = fsa_latlon.merge(fsa_income,left_on='FSA', right_on='GEO_NAME', how='left')

In [6]:
fsa_facts=fsa_facts[['FSA','lat','lon','DIM', 'DIM_ID', 'Value']]

In [7]:
# def find_first_2_letters(row):
#     return row['FSA'][0:2]

# fsa_facts['FSA_2L'] = fsa_facts.apply(find_first_2_letters, axis=1)

In [12]:
fsa_facts['Value']=pd.to_numeric(fsa_facts['Value'], errors='coerce')
fsa_facts.dropna(inplace=True)

In [13]:
fsa_facts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 57
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   FSA     55 non-null     object 
 1   lat     55 non-null     float64
 2   lon     55 non-null     float64
 3   DIM     55 non-null     object 
 4   DIM_ID  55 non-null     int64  
 5   Value   55 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 3.0+ KB


In [23]:
from sklearn import preprocessing

x = fsa_facts['Value'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

fsa_facts['value_norm'] = x_scaled

In [27]:
m = folium.Map(
    location=[fsa_latlon['lat'].mean(), fsa_latlon['lon'].mean()],
    zoom_start=10,
    tiles='Stamen Terrain'
)


for i,row in fsa_facts.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row['value_norm']*10,
        color='#3186cc',
        popup=[row['FSA'],row['Value']],
        fill=True,
        fill_color='#3186cc'
    ).add_to(m)

m